In [3]:
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline
import pandas as pd
import pymysql

model_path = "bert-base-chinese-finetuning-financial-news-sentiment"

model=AutoModel.from_pretrained(model_path)
tokenizer=AutoTokenizer.from_pretrained(model_path)
pipe = pipeline("text-classification", model=model_path,device=0)

conn = pymysql.connect(host='localhost', user='root', password='123456', database='citicup', charset='utf8')

# 创建游标
cursor = conn.cursor()

In [4]:
def getScore(input):
    response = pipe(input)
    score = 0
    weights = [0.6, 0.3, 0.1] 
    for index, i in enumerate(response):
        weight = weights[index] if index < len(weights) else 0
        if i['label'] == 'Positive':
            score += 1 * weight
        elif i['label'] == 'Negative':
            score += -1 * weight
        # print(i['label'], score)
    return score

In [6]:
df = pd.read_excel('ClimbingResearchTitle.xls')
inputs = []

insert_sql = "INSERT INTO research_title_score (stock_code, score) VALUES (%s, %s)"

data_to_insert = []
for i, row in df.iterrows():
    inputs.append(row['post_title'])
    if (i + 1) % 3 == 0:
        stock_code = str(row['stock_code']).zfill(6) 
        score = getScore(inputs)
        score = "{:.1f}".format(score)
        data_to_insert.append((stock_code, score))
        inputs = []
        if len(data_to_insert) >= 1000:
            cursor.executemany(insert_sql, data_to_insert)
            conn.commit()
            data_to_insert = []
if data_to_insert:
    cursor.executemany(insert_sql, data_to_insert)
    conn.commit()

cursor.close()
conn.close()

d:\anaconda\envs\test\lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
import csv
import pymysql

cnx = pymysql.connect(host='localhost', user='root', password='123456', database='citicup', charset='utf8')

try:
    with cnx.cursor() as cursor:
        cursor.execute("SELECT * FROM research_title_score")
        column_names = [i[0] for i in cursor.description]
        with open('research_title_score.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(column_names)
            for row in cursor: 
                writer.writerow(row)
finally:
    cnx.close()


In [5]:
import pymysql
import pickle
cnx = pymysql.connect(host='localhost', user='root', password='123456', database='citicup', charset='utf8')

try:
    with cnx.cursor() as cursor:
        cursor.execute("SELECT * FROM title_score")
        column_names = [i[0] for i in cursor.description]
        rows = cursor.fetchall()
        data = {
            'column_names': column_names,
            'data': rows
        }
        with open('title_score.pkl', 'wb') as pklfile:
            pickle.dump(data, pklfile)

finally:
    cnx.close()


In [4]:
with open('research_title_score.pkl', 'rb') as pklfile:
    data = pickle.load(pklfile)
print(data['data'])

(('000001', '1.0'), ('000002', '-0.2'), ('000005', '0.6'), ('000006', '1.0'), ('000007', '0.1'), ('000008', '0.9'), ('000010', '0.1'), ('000012', '1.0'), ('000016', '0.6'), ('000021', '-0.2'), ('000025', '-0.6'), ('000026', '1.0'), ('000027', '0.9'), ('000028', '1.0'), ('000030', '1.0'), ('000031', '-0.2'), ('000032', '0.4'), ('000034', '0.7'), ('000035', '0.1'), ('000036', '0.3'), ('000039', '1.0'), ('000040', '0.4'), ('000042', '0.6'), ('000045', '0.9'), ('000048', '0.8'), ('000049', '0.7'), ('000050', '1.0'), ('000055', '0.9'), ('000056', '1.0'), ('000059', '0.1'), ('000060', '0.8'), ('000061', '0.6'), ('000062', '1.0'), ('000063', '0.0'), ('000065', '0.4'), ('000066', '1.0'), ('000068', '0.1'), ('000069', '0.4'), ('000070', '0.9'), ('000078', '0.4'), ('000088', '1.0'), ('000089', '1.0'), ('000090', '1.0'), ('000099', '1.0'), ('000100', '-0.2'), ('000153', '0.3'), ('000155', '0.7'), ('000156', '0.9'), ('000157', '0.7'), ('000158', '0.6'), ('000166', '1.0'), ('000301', '0.8'), ('0003